In [ ]:
import numpy as np
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from math import sqrt
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
import copy
from collections import defaultdict
import matplotlib.pyplot as plt
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [ ]:
def extract_data(path,create_test=None):
    X_train=[]
    y_train=[]
    X_test=[]
    y_test=[]

    file=os.listdir(path)

    for entity in file:
        try:
            entityPath=os.listdir(f"{path}/{entity}")
            for i,data in enumerate(entityPath):
                try:
                    current_img=cv2.imread(f"{path}/{entity}/{data}")
                    if current_img is not None:
                        if (create_test is not None and ((len(entityPath)>1) and (i==0))):
                            X_test.append(current_img)
                            y_test.append(entity)
                        else:
                            X_train.append(current_img)
                            y_train.append(entity)
                    else:
                        print(f"{path}/{entity}/{data} is None type")
                except Exception as e:
                    print(f"Erreur avec {path}/{entity}/{data} : {e}")
        except Exception as ex:
            print(f"Erreur avec {path}/{entity} : {ex}")
    
    print(f"Number of training sample : {len(X_train)}\n")
    if create_test is not None:
        print(f"Number of test sample : {len(X_test)}\n")
        return X_train,y_train,X_test,y_test
    else:
        return X_train,y_train

In [ ]:
transform_test=transforms.Compose([
    transforms.ToPILImage(), #transforme en format PIL 
    transforms.ToTensor() #reconvertit l'img en format tensor
])

transform=transforms.Compose([
    transforms.ToPILImage(), #transforme en format PIL
    transforms.RandomHorizontalFlip(), #0.5 de proba d'inverser la gauche et la droite de l'img pour rendre le modèle invariant à la symétrie
    transforms.RandomRotation(15), #applique rota random entre -10° et +10° 
    transforms.ColorJitter(brightness=0.3,contrast=0.3,saturation=0.2,hue=0.02), #altère aléatoirement la luminosité et le contraste de l'image
    transforms.ToTensor() #reconvertit l'img en format tensor
])

In [ ]:
def imgProcess(img,withTorch=None,tr=True):
    if not isinstance(img,np.ndarray):
        print("convertion de l'img")
        img=cv2.imread(img)

    img=cv2.resize(img,(100,100))
    if withTorch is None:
        img=img.flatten()
    else:
        if tr is not None:
            img=transform(img)
        else:
            img=transform_test(img)
        img=img.unsqueeze(0)
    return img

In [ ]:
def extract_face_and_eyes(img):
    if not isinstance(img,np.ndarray):
        img=cv2.imread(img)

    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(60,60)
    )

    if len(faces)==0:
        return [],[]
    
    x,y,w,h=faces[0]
    face=img[y:y+h,x:x+w]
    eyes=eye_cascade.detectMultiScale(cv2.cvtColor(face,cv2.COLOR_BGR2GRAY))
    if len(eyes)==2:
        if eyes[0][0]<eyes[1][0]:
            x2=eyes[0][0]
            w2=eyes[1][0]+eyes[1][2]-x2
        else:
            x2=eyes[1][0]
            w2=eyes[0][0]+eyes[0][2]-x2
        if eyes[0][1]<eyes[1][1]:
            y2=eyes[0][1]
            h2=eyes[1][1]+eyes[1][3]-y2
        else:
            y2=eyes[1][1]
            h2=eyes[0][1]+eyes[0][3]-y2
        imgEyes=face[y2:y2+h2,x2:x2+w2]
        return face,imgEyes
    else:
        return face,[]

In [ ]:
def createData(X,y):
    x_face=[]
    y_face=[]
    x_eyes=[]
    y_eyes=[]
    for i,x in enumerate(X):
        face,eyes=extract_face_and_eyes(x)
        if len(face)!=0:
            x_face.append(face)
            y_face.append(y[i])
        if len(eyes)!=0:
            x_eyes.append(eyes)
            y_eyes.append(y[i])
    return x_face,y_face,x_eyes,y_eyes

In [ ]:
def make_face_and_eyes_models(nb_classes):
    face_model=nn.Sequential(

    )

    eyes_model=nn.Sequential(

    )

    final_classifier=nn.Sequential(
        nn.Linear(),
        nn.ReLU(),
        #nn.Dropout(),
        nn.Linear(,nb_classes)
    )

    return face_model,eyes_model,final_classifier

In [ ]:
def combine_models(face_model,eyes_model,face_img,eyes_img)

In [ ]:
def CNN_train(X_train,y_train,nb_epoch,labels,batch_size,models=None,patience=5,val_split=None):
    if models is None:
        face_model,eyes_model,final_classifier=make_face_and_eyes_models(len(list(set(y_train))))
    else:
        face_model,eyes_model,final_classifier=models

    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    face_model.to(device)
    eyes_model.to(device)
    final_classifier.to(device)

    if val_split is not None:
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train,random_state=1)
        val_dataset = list(zip(X_val, [labels[y] for y in y_val]))
        val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False, collate_fn=lambda x: x)
        best_val_acc = 0
        wait = 0

    y_train_tensor=torch.tensor([labels[y] for y in y_train])

    dataset=list(zip(X_train,y_train_tensor))
    dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=True,collate_fn=lambda x: x)

    criterion = torch.nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    tabTrain=[]
    tabVal=[]

    model.train()
    for epoch in range(nb_epoch):
        total_loss=0.0
        correct=0
        total=0
        for batch in dataloader:
            X_raw,labels_batch=zip(*batch)
            img=[imgProcess(image,withTorch=True) for image in X_raw]
            img=torch.cat(img,dim=0)
            img=img.to(device)
            labelsT=torch.tensor(labels_batch).to(device)

            outputs=model(img)
            loss=criterion(outputs,labelsT)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()

            total_loss+=loss.item()
            _,predicted=torch.max(outputs,1)
            correct+=(predicted==labelsT).sum().item()
            total+=labelsT.size(0)
            
        acc=100*correct/total
        tabTrain.append(acc)
        print(f"\nEpoch : {epoch+1}/{nb_epoch}\n Perte :{total_loss:.4f}\n Précision : {acc:.2f}%")
        if val_split is not None:
            true,false,tot=CNN_evaluate(model=model,labels=labels,loader=val_loader,device=device)
            val_acc=sum(true.values())/sum(tot.values())
            tabVal.append(100*val_acc)
            if val_acc>best_val_acc:
                best_val_acc=val_acc
                wait=0
                best_model=copy.deepcopy(model.state_dict())
            else:
                wait+=1
                if wait>=patience:
                    print("Early stopping activated")
                    break
            print(f"\n Val accuracy : {val_acc*100:.2f}%\n Série sans amélioration : {wait}")
            model.load_state_dict(best_model)
        
    absc=list(range(len(tabTrain)))
    plt.figure(figsize=(8,5))
    plt.plot(absc,tabTrain,label="Training",marker="o")
    if val_split is not None:
        plt.plot(absc,tabVal,label="Validation",marker="s")
    plt.xlim(1,len(tabTrain))
    plt.ylim(0,100)
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy (%)")
    plt.legend()
    plt.grid(True)
    print("Training terminé.")
    plt.show()
    return model

In [ ]:
X_train,y_train=extract_data("DataFaces")
X_train,X_test,y_train,y_test=train_test_split(X_train,y_train,test_size=0.2,random_state=1,stratify=y_train)
X_train_faces,y_train_faces,X_train_eyes,y_train_eyes=createData(X_train,y_train)
X_test_faces,y_test_faces,X_test_eyes,y_test_eyes=createData(X_test,y_test)
labels={label:i for i,label in enumerate(sorted(set(y_train)))} #list(set(y_train)) sort les val uniques en list et sorted les range dans l'ordre alphabétique
print(f"Face train samples : {len(X_train_faces)}.\nEyes train samples : {len(X_train_eyes)}.\nFace test samples : {len(X_test_faces)}.\nEyes test samples : {len(X_test_eyes)}.\n")